In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, ZeroPadding2D,Flatten, DepthwiseConv2D, BatchNormalization, ReLU, Add, Dense, GlobalAveragePooling2D, Input, Dropout, MaxPooling2D, AveragePooling2D, Concatenate




from sklearn.model_selection import train_test_split
import cv2

import matplotlib.pyplot as plt

In [2]:
path = "/kaggle/input/yasheen-khan/PCG_data"
categories = ['N', 'MR', 'MVP', 'AS', 'MS']
sample_file = "/kaggle/input/yasheen-khan/PCG_data/AS/New_AS_001.wav"
sr = 4000
n_fft = 2048
hop_length = 512
img_size = (128, 128)

def extract_stft(file):
    signal, _ = librosa.load(file, sr=sr)
    stft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    spectrogram = cv2.resize(spectrogram, img_size)
    spectrogram = np.stack([spectrogram] * 3, axis=-1)
    return np.array(spectrogram)

features = []
labels = []
for idx,lable in enumerate(categories):
    dir_ = f"{path}/{lable}/"
    for file in os.listdir(dir_):
        if file.endswith(".wav"):
            file = dir_+file
            features.append(extract_stft(file))
            labels.append(idx)

features = np.array(features, dtype="float32") / 255.0 
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(categories))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(categories))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((800, 128, 128, 3), (800, 5), (200, 128, 128, 3), (200, 5))

# With Inseption

In [3]:
def add_block(x, filters, padding):
    # F1,K1
    x = Conv2D(filters, (3,3), padding ='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    #F2,K2
    x = Conv2D(filters, (3,3), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # F3, K3
    x = Conv2D(2*filters, (3,3), padding = padding)(x)
    x = BatchNormalization()(x)
    
    return x


def create_model():
    input_layer = Input(shape = (128,128,3))
    x = Conv2D(32, (5,5), padding='valid')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x0 = MaxPooling2D(2,2)(x)

    block1 = add_block(x0, filters = 64, padding = 'valid')
    alt1 = BatchNormalization()(Conv2D(128, (3,3))(x0))

    x1 = Add()([block1, alt1])

    x = ReLU()(x1)
    block2 = add_block(x, filters = 128, padding = 'same')
    alt2 = Conv2D(256,(1,1), padding = 'same')(ReLU()(x1))
    # alt2 = ZeroPadding2D(((10, 10), (10, 10)))(x)
    
    x2 = Add()([block2, alt2])

    x = ReLU()(x2)
    x = Flatten()(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(5, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=x)
    
    
model = create_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 124, 124, 32)   │          2,432 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 124, 124, 32)   │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 124, 124, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 62, 62, 32)     │              0 │ re_lu[0][0]            │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 62, 62, 64)     │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 62, 62, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 62, 62, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 62, 62, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 62, 62, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 62, 62, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 60, 60, 128)    │         73,856 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 60, 60, 128)    │         36,992 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 60, 60, 128)    │            512 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 60, 60, 128)    │            512 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 60, 60, 128)    │              0 │ batch_normalization_3… │
│                           │                        │                │ batch_normalization_4… │
├──────────────────────

 Total params: 472,658,053 (1.76 GB)

 Trainable params: 472,656,197 (1.76 GB)

 Non-trainable params: 1,856 (7.25 KB)

In [4]:
# model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=, batch_size=32, validation_data=(X_test,y_test))

In [5]:
def add_block(x, filters, padding):
    # F1,K1
    x = Conv2D(filters, (3,3), padding ='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    #F2,K2
    x = Conv2D(filters, (3,3), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # F3, K3
    x = Conv2D(2*filters, (3,3), padding = padding)(x)
    x = BatchNormalization()(x)
    
    return x


def inception_block(x):
    x0 = Conv2D(64, (1,1), padding="same")(x)

    x1 = Conv2D(96, (1,1), padding="same")(x)
    x1 = Conv2D(128, (3,3), padding = "same")(x1)

    x2 = Conv2D(16, (1,1), padding="same")(x)
    x2 = Conv2D(32, (5,5), padding = "same")(x2)
    
    x3 = MaxPooling2D((3,3), strides=1, padding="same")(x)
    x3 = Conv2D(128, (3,3), padding = "same")(x3)

    x = Concatenate(axis=-1)([x0,x1,x2,x3])
    return x
    
    
def create_model():
    input_layer = Input(shape = (128,128,3))
    x = Conv2D(32, (5,5), padding='valid')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x0 = MaxPooling2D(2,2)(x)

    block1 = add_block(x0, filters = 64, padding = 'valid')
    alt1 = BatchNormalization()(Conv2D(128, (3,3))(x0))

    x1 = Add()([block1, alt1])

    x = inception_block(x1)
    block2 = add_block(x, filters = 128, padding = 'same')
    alt2 = Conv2D(256,(1,1), padding = 'same')(ReLU()(x1))
    # alt2 = ZeroPadding2D(((10, 10), (10, 10)))(x)
    
    x2 = Add()([block2, alt2])

    x = inception_block(x2)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(5, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=x)
    
    
model = create_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 124, 124, 32)   │          2,432 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_8     │ (None, 124, 124, 32)   │            128 │ conv2d_9[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_8 (ReLU)            │ (None, 124, 124, 32)   │              0 │ batch_normalization_8… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 62, 62, 32)     │              0 │ re_lu_8[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 62, 62, 64)     │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_9     │ (None, 62, 62, 64)     │            256 │ conv2d_10[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_9 (ReLU)            │ (None, 62, 62, 64)     │              0 │ batch_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 62, 62, 64)     │         36,928 │ re_lu_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_10    │ (None, 62, 62, 64)     │            256 │ conv2d_11[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_10 (ReLU)           │ (None, 62, 62, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 60, 60, 128)    │         73,856 │ re_lu_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 60, 60, 128)    │         36,992 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_11    │ (None, 60, 60, 128)    │            512 │ conv2d_12[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 60, 60, 128)    │            512 │ conv2d_13[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 60, 60, 128)    │              0 │ batch_normalization_1… │
│                      

 Total params: 1,994,789 (7.61 MB)

 Trainable params: 1,992,933 (7.60 MB)

 Non-trainable params: 1,856 (7.25 KB)

In [6]:
# model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=, batch_size=32, validation_data=(X_test,y_test))